## Resampling with Rioxarray (S3 storage, NetCDF File, NetCDF4 driver, VSIS3 virtualization, earthaccess auth)

In [ ]:
import argparse
import sys

import rasterio as rio
import rioxarray as rx
from rasterio.session import AWSSession
from rasterio.warp import calculate_default_transform

In [ ]:
sys.path.append("..")

In [ ]:
def configure_auth(daac):
    """Authenticate using earthaccess"""
    import boto3
    import earthaccess

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials(daac)
    session = boto3.Session(
        aws_access_key_id=s3_credentials["accessKeyId"],
        aws_secret_access_key=s3_credentials["secretAccessKey"],
        aws_session_token=s3_credentials["sessionToken"],
        region_name="us-west-2",
    )
    rio_env = rio.Env(
        AWSSession(session),
    )
    rio_env.__enter__()
    return rio_env

In [ ]:
def warp_resample(dataset):
    from common import earthaccess_args
    from common import target_extent as te

    # Define filepath, driver, and variable information
    args = earthaccess_args[dataset]
    input_uri = f'{args["folder"]}/{args["filename"]}'
    src = f'NETCDF:/vsis3/{args["bucket"]}/{input_uri}:{args["variable"]}'
    # Define source and target projection
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    with configure_auth(args["daac"]):
        # Open dataset
        da = rx.open_rasterio(src, mask_and_scale=True)
        # Set input dataset projection
        da = da.rio.write_crs(srcSRS)
        da = da.rio.clip_box(
            *te,
            crs=dstSRS,
        )
        # Define affine transformation from input to output dataset
        dst_transform, w, h = calculate_default_transform(
            srcSRS,
            dstSRS,
            da.rio.width,
            da.rio.height,
            *da.rio.bounds(),
            dst_width=width,
            dst_height=height,
        )
        # Reproject dataset
        return da.rio.reproject(dstSRS, shape=(h, w), transform=dst_transform)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        # Just call warp_resample if running as a Jupyter Notebook
        da = warp_resample("mursst")
    else:
        # Configure dataset via argpase if running via CLI
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["mursst"],
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset)